In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib2
from sklearn import linear_model
import numpy as np
import itertools

In [ ]:
## We will find all listings with word concert, probably ticket do buy (but not always)
BASE_URL = ('http://chicago.craigslist.org/search/sss?query={}&sort=rel')
search_term = "concert"
results = []
search_term = search_term.strip().replace(' ', '+')
search_url = BASE_URL.format(search_term)
soup = BeautifulSoup(urllib2.urlopen(search_url).read())

In [ ]:
soup.prettify

EX. 3

zrobione :)

EX. 4

In [4]:
df = pd.read_csv("filmdeathcounts.csv")
df.head()

,Film,Year,Body_Count,MPAA_Rating,Genre,Director,Length_Minutes,IMDB_Rating
0,24 Hour Party People,2002,7,R,Biography|Comedy|Drama|Music,Michael Winterbottom,117,7.3
1,28 Days Later,2002,53,R,Horror|Sci-Fi|Thriller,Danny Boyle,113,7.6
2,28 Weeks Later,2007,212,R,Horror|Sci-Fi|Thriller,Juan Carlos Fresnadillo,100,7.0
3,30 Days of Night,2007,67,R,Horror|Thriller,David Slade,113,6.6
4,300,2007,600,R,Action|Fantasy|History|War,Zack Snyder,117,7.7


In [5]:
genres_chain = itertools.chain(*[x.split("|") for x in df["Genre"]])
genres = list(set(genres_chain))
print genres

['Sci-Fi', 'Crime', 'Romance', 'Animation', 'Music', 'Comedy', 'War', 'Horror', 'Film-Noir', 'Adventure', 'Thriller', 'Western', 'Mystery', 'Drama', 'Action', 'Documentary', 'Musical', 'History', 'Family', 'Fantasy', 'Sport', 'Biography']


In [6]:
for genre in genres:
    l = [(int)(x.find(genre) != -1) for x in df["Genre"]]
    df[genre] = l
df.head()

,Film,Year,Body_Count,MPAA_Rating,Genre,Director,Length_Minutes,IMDB_Rating,Sci-Fi,Crime,...,Mystery,Drama,Action,Documentary,Musical,History,Family,Fantasy,Sport,Biography
0,24 Hour Party People,2002,7,R,Biography|Comedy|Drama|Music,Michael Winterbottom,117,7.3,0,0,...,0,1,0,0,0,0,0,0,0,1
1,28 Days Later,2002,53,R,Horror|Sci-Fi|Thriller,Danny Boyle,113,7.6,1,0,...,0,0,0,0,0,0,0,0,0,0
2,28 Weeks Later,2007,212,R,Horror|Sci-Fi|Thriller,Juan Carlos Fresnadillo,100,7.0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,30 Days of Night,2007,67,R,Horror|Thriller,David Slade,113,6.6,0,0,...,0,0,0,0,0,0,0,0,0,0
4,300,2007,600,R,Action|Fantasy|History|War,Zack Snyder,117,7.7,0,0,...,0,0,1,0,0,1,0,1,0,0


In [7]:
important_cols = [2, 3] + range(8, len(df.columns))
data = df.ix[:, important_cols]
data["MPAA_Rating"] = data["MPAA_Rating"] == "R"
data.head()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Body_Count,MPAA_Rating,Sci-Fi,Crime,Romance,Animation,Music,Comedy,War,Horror,...,Mystery,Drama,Action,Documentary,Musical,History,Family,Fantasy,Sport,Biography
0,7,True,0,0,0,0,1,1,0,0,...,0,1,0,0,0,0,0,0,0,1
1,53,True,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,212,True,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,67,True,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,600,True,0,0,0,0,0,0,1,0,...,0,0,1,0,0,1,0,1,0,0


In [8]:
print sum(data["MPAA_Rating"])
print len(data)

338
545


In [9]:
x_cols = list(data.columns[0:1] + data.columns[2:])
print x_cols

['Action', 'Adventure', 'Animation', 'Biography', 'Body_Count', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: using '+' to provide set union with Indexes is deprecated, use '|' or .union()
  if __name__ == '__main__':


In [10]:
logreg = linear_model.LogisticRegression(C=1e5)

In [11]:
rs = range(len(data))
np.random.shuffle(rs)
train_size = int(len(data) * 0.666)
test_size = len(data) - train_size

In [12]:
train = data.iloc[rs[:train_size]]
test = data.iloc[rs[-test_size:]]

In [13]:
X = train[x_cols]
Y = train["MPAA_Rating"]
assert len(X) == len(Y)

In [14]:
logreg.fit(X, Y)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [15]:
Z = logreg.predict(test[x_cols])
YY = test["MPAA_Rating"]
print "accuracy = {}%".format(100.0 * float(sum(a == b for a, b in zip(YY,Z))) / len(Z))

accuracy = 69.9453551913%
